<center></center>
<img src="../__data/logo.png" alt="Mohammadkh.ir" style="width: 250px;"/>
<h1><center><div style="direction:rtl">Open CV</div></center></h1>

### paper --> http://richzhang.github.io/colorization/

In [1]:
import cv2
import numpy as np

In [2]:
proto_file = '../__data/Detection model/gray to color/colorization_deploy_v2.prototxt'
model_file = '../__data/Detection model/gray to color/colorization_release_v2.caffemodel'
hull_pts = '../__data/Detection model/gray to color/pts_in_hull.npy'

net = cv2.dnn.readNetFromCaffe(proto_file,model_file)
kernel = np.load(hull_pts)

class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = kernel.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

In [3]:
img_path = '../__data/image/gray.jpg'

img = cv2.imread(img_path)
# scale
scaled = img.astype("float32") / 255.0
# conv bgr to LAB
lab_img = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)
# resize to input network
resized = cv2.resize(lab_img, (224, 224))
# split the L channel
L = cv2.split(resized)[0]
# mean subtraction
L -= 50

# peredict
net.setInput(cv2.dnn.blobFromImage(L))
ab_channel = net.forward()
ab_channel = ab_channel[0, :, :, :].transpose((1, 2, 0))
# resize the predicted 'ab' volume to the same dimensions as our
# input image
ab_channel = cv2.resize(ab_channel, (img.shape[1], img.shape[0]))


# Take the L channel from the image
L = cv2.split(lab_img)[0]
# Join the L channel with predicted ab channel
colorized = np.concatenate((L[:, :, np.newaxis], ab_channel), axis=2)

# Then convert the image from Lab to BGR
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
# all val between to 0-1
colorized = np.clip(colorized, 0, 1)
# change the image to 0-255 range and convert it from float32 to int
colorized = (255 * colorized).astype("uint8")

# Let's resize the images and show better them together
img = cv2.resize(img,(640,640))
colorized = cv2.resize(colorized,(640,640))

result = cv2.hconcat([img,colorized])

cv2.imshow("Grayscale -> Colour", result)

cv2.waitKey(0)
cv2.destroyAllWindows()

<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:left"><strong> OpenCV</strong><br>MohammadReza <strong>Khajedaloi</strong><br><br>
</div>
<div style="direction:rtl;text-align:right">
<a href="http://mohammadkh.ir/">WebSite</a> - <a href="https://github.com/khajedaloi/">GitHub</a> - <a href="https://www.linkedin.com/in/mohammad-kh/">Linkedin</a>
</div>
</div>